### Deep Learning Barrier Option

We used Numba and CuPy in the previous notebook to run Monte Carlo simulation to determine the price of the Asian Barrier option. A Monte Carlo simulation needs millions of paths to get an accurate answer which is computationally intensive. [Ryan et al (2018)](https://arxiv.org/abs/1809.02233) showed that a deep learning model can be trained to value derivatives. The deep learning model is accurate and very fast, capable of producing valuations a million times faster than traditional models. In the this notebook, we will use a fully connected network to learn the pricing mode of the Asian Barrier option. Monte Carlo simulation is used as pricing ground truth for the training. We use the same Asian Barrier Option model as last notebook with parameters listed as following:

```
T - Maturity (yrs.)
S - Spot (usd)
K - Strike (usd)
sigma - Volatility (per.)
r - Risk Free Rate (per.)
mu - Stock Drift Rate (per.)
B - Barrier (usd)
```

### Batched Data generation

The dataset is an important part of the Deep learning training. We will modify the previous single Asian Barrier Option pricing code to handle a batch of Barrier Option pricing. 

Loading all the necessary libraries:-

In [1]:
!curl https://colab.chainer.org/install |sh -
import cupy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   5249      0 --:--:-- --:--:-- --:--:--  5266
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9MB 81kB/s 
+ set +ex
Installation succeeded!


In [2]:
import cupy
import numpy as np
import math
import time
import torch
cupy.cuda.set_allocator(None)
from torch.utils.dlpack import from_dlpack

The CuPy version of batched barrier option pricing simulation is as follows:-

In [ ]:
cupy_batched_barrier_option = cupy.RawKernel(r'''
extern "C" __global__ void batched_barrier_option(
    float *d_s,
    const float T,
    const float * K,
    const float * B,
    const float * S0,
    const float * sigma,
    const float * mu,
    const float * r,
    const float * d_normals,
    const long N_STEPS,
    const long N_PATHS,
    const long N_BATCH)
{
  unsigned idx =  threadIdx.x + blockIdx.x * blockDim.x;
  unsigned stride = blockDim.x * gridDim.x;
  unsigned tid = threadIdx.x;
  const float tmp3 = sqrt(T/N_STEPS);


  for (unsigned i = idx; i<N_PATHS * N_BATCH; i+=stride)
  {
    int batch_id = i / N_PATHS;
    int path_id = i % N_PATHS;
    float s_curr = S0[batch_id];
    float tmp1 = mu[batch_id]*T/N_STEPS;
    float tmp2 = exp(-r[batch_id]*T);
    unsigned n=0;
    double running_average = 0.0;
    for(unsigned n = 0; n < N_STEPS; n++){
       s_curr += tmp1 * s_curr + sigma[batch_id]*s_curr*tmp3*d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH];
       running_average += (s_curr - running_average) / (n + 1.0);
       if (running_average <= B[batch_id]){
           break;
       }
    }

    float payoff = (running_average>K[batch_id] ? running_average-K[batch_id] : 0.f); 
    d_s[i] = tmp2 * payoff;
  }
}

''', 'batched_barrier_option')

Note, the parameters (K, B, S0, sigma, mu, r) are passed in as an array with length of batch size. The output array is a two dimensional array flatten to 1-D. The first dimension is for Batch and the second dimension is for Path. 

Testing it out by entering two sets of option parameters:-

In [ ]:
N_PATHS = 204800
N_STEPS = 365
N_BATCH = 2
T = 1.0

K = cupy.array([110.0, 120.0], dtype=cupy.float32)
B = cupy.array([100.0, 90.0], dtype=cupy.float32)
S0 = cupy.array([120.0, 100.0], dtype=cupy.float32)
sigma = cupy.array([0.35, 0.2], dtype=cupy.float32)
mu = cupy.array([0.15, 0.1], dtype=cupy.float32)
r =cupy.array([0.05, 0.05], dtype=cupy.float32)


Put everything into a simple function to launch this GPU kernel. The option prices for each batch is the average of the corresponding path terminal values. This can be computed easily by Cupy function `mean(axis=1)`

In [ ]:
############ new
batch_randoms_mean = np.zeros(2) # assume 2 options
batch_randoms_cov = [[1, 0],[0, 1]]
num_of_randoms_each_batch = N_PATHS * N_STEPS
############ new

def batch_run():
    number_of_threads = 256
    number_of_blocks = (N_PATHS * N_BATCH - 1) // number_of_threads + 1
    #randoms_gpu = cupy.random.normal(0, 1, N_BATCH*N_PATHS * N_STEPS, dtype=cupy.float32)

    ############ new
    randoms_gpu = cupy.random.multivariate_normal(batch_randoms_mean, batch_randoms_cov,
                                                  num_of_randoms_each_batch, dtype=cupy.float32)
    randoms_gpu = np.transpose(randoms_gpu).reshape(1,-1)[0]
    ############ new

    output = cupy.zeros(N_BATCH*N_PATHS, dtype=cupy.float32)
    cupy.cuda.stream.get_current_stream().synchronize()
    s = time.time()
    cupy_batched_barrier_option((number_of_blocks,), (number_of_threads,),
                       (output, np.float32(T), K, B, S0, sigma, mu, r,
                        randoms_gpu, N_STEPS, N_PATHS, N_BATCH))
    v = output.reshape(N_BATCH, N_PATHS).mean(axis=1)
    cupy.cuda.stream.get_current_stream().synchronize()
    e = time.time()
    print('time', e-s, 'v',v)
batch_run()

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


KeyboardInterrupt: ignored

In [ ]:
############ new - for more options (say 50), just provide mean with dim 1x50 and cov with dim 50x50
batch_randoms_mean = np.zeros(2) # assume 2 options
batch_randoms_cov = [[1, -0.99],[-0.99, 1]]
num_of_randoms_each_batch = N_PATHS * N_STEPS
############ new

def batch_run():
    number_of_threads = 256
    number_of_blocks = (N_PATHS * N_BATCH - 1) // number_of_threads + 1
    #randoms_gpu = cupy.random.normal(0, 1, N_BATCH*N_PATHS * N_STEPS, dtype=cupy.float32)

    ############ new - add correlation between batches
    all_normals = cupy.random.multivariate_normal(batch_randoms_mean, batch_randoms_cov, 
                                          N_PATHS * N_STEPS, dtype=cupy.float32)
    randoms_gpu = all_normals[:,0].reshape(-1,N_PATHS)

    for i in range(1,N_BATCH):
      tempdf = all_normals[:,i].reshape(-1,N_PATHS)
      randoms_gpu = cupy.concatenate((randoms_gpu, tempdf), axis = 1)

    randoms_gpu = randoms_gpu.reshape(1,-1)[0]
    ############ new

    output = cupy.zeros(N_BATCH*N_PATHS, dtype=cupy.float32)
    cupy.cuda.stream.get_current_stream().synchronize()
    s = time.time()
    cupy_batched_barrier_option((number_of_blocks,), (number_of_threads,),
                       (output, np.float32(T), K, B, S0, sigma, mu, r,
                        randoms_gpu, N_STEPS, N_PATHS, N_BATCH))
    v = output.reshape(N_BATCH, N_PATHS).mean(axis=1)
    cupy.cuda.stream.get_current_stream().synchronize()
    e = time.time()
    print('time', e-s, 'v',v)
batch_run()

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


time 0.002972841262817383 v [21.29957     0.84790635]


In [ ]:
for i in np.arange(-0.99, 0.99, 0.1):
  batch_randoms_cov = [[1, i],[i, 1]]
  batch_run()

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


time 0.002907276153564453 v [21.30244    0.8401122]
time 0.0030074119567871094 v [21.206484    0.83364785]
time 0.0029032230377197266 v [21.098728    0.86061645]
time 0.0028994083404541016 v [21.208979    0.84774697]
time 0.002958059310913086 v [21.28604    0.8369874]
time 0.002973318099975586 v [21.271456   0.8391292]
time 0.0029020309448242188 v [21.315578   0.8359961]
time 0.0029685497283935547 v [21.355581   0.8517328]
time 0.0029726028442382812 v [21.237495   0.8452696]
time 0.0029947757720947266 v [21.123238   0.8491896]
time 0.0028955936431884766 v [21.160463   0.8498547]
time 0.0029120445251464844 v [21.233788   0.8522683]


KeyboardInterrupt: ignored

In [ ]:
np.arange(-0.99, 0.99, 0.1)

array([-0.99, -0.89, -0.79, -0.69, -0.59, -0.49, -0.39, -0.29, -0.19,
       -0.09,  0.01,  0.11,  0.21,  0.31,  0.41,  0.51,  0.61,  0.71,
        0.81,  0.91])

In [ ]:
test_N_PATHS = 5
test_N_STEPS = 2000
test_N_BATCH = 2

test = cupy.random.multivariate_normal([0,0], [[1,0.5],[0.5,1]], test_N_PATHS * test_N_STEPS, dtype=cupy.float32)
test

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


array([[ 0.36617985,  1.0074509 ],
       [ 2.21951   ,  1.1886001 ],
       [ 0.6720402 ,  0.9284749 ],
       ...,
       [ 0.72382426, -0.79330003],
       [-0.8009655 ,  0.43871942],
       [-0.62982166,  0.6527443 ]], dtype=float32)

In [ ]:
print(test[:,0])
print(test[:,1])

[-1.3275453  -1.7941357  -0.39915624 ... -0.28993356 -0.7993693
 -0.578034  ]
[-1.7857952  -2.086197   -1.4743258  ... -1.4586484  -1.0170573
 -0.51631546]


In [ ]:
np.cov(test[:,0], test[:,1])

array([[1.00962119, 0.49988681],
       [0.49988681, 1.00435632]])

In [ ]:
# -1 means unknown -> we need it to be one row
np.transpose(test).reshape(1,-1)[0]

array([-1.3275453 , -1.7941357 , -0.39915624, ..., -1.4586484 ,
       -1.0170573 , -0.51631546], dtype=float32)

In [ ]:
test_randoms = cupy.empty(0)
for i in range(test_N_STEPS):
  temp = cupy.random.multivariate_normal([0,0], [[1,0.5],[0.5,1]],
                                  test_N_PATHS, dtype=cupy.float32)
  test_randoms = cupy.append(test_randoms, cupy.transpose(temp).reshape(1,-1)[0])

test_randoms

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


array([-0.55608845, -0.50560349, -0.31312737, ...,  0.46078965,
       -1.56287372,  2.99057341])

In [ ]:
test = cupy.random.multivariate_normal([0,0], [[1,0.5],[0.5,1]], 
                                       test_N_PATHS * test_N_STEPS, dtype=cupy.float32)

test_randoms = test[:,0].reshape(-1,test_N_PATHS)

for i in range(1,test_N_BATCH):
  tempdf = test[:,i].reshape(-1,test_N_PATHS)
  test_randoms = cupy.concatenate((test_randoms, tempdf), axis = 1)


In [ ]:
test_randoms

array([[-0.15535657,  0.31599703,  1.9873533 , ...,  0.5068042 ,
         0.49160695, -1.2550368 ],
       [ 0.43182704, -1.3187023 ,  0.0042416 , ..., -0.45345792,
         0.321936  ,  1.0028907 ],
       [-1.064633  , -1.0312779 , -0.5822    , ...,  0.09054348,
        -0.80207705, -0.22503746],
       ...,
       [ 1.1523024 ,  0.31871992,  0.38878578, ...,  0.1350182 ,
         1.156924  , -0.41682786],
       [-0.31418055, -1.366479  , -0.81390893, ..., -0.02646282,
        -0.0627628 , -2.1372068 ],
       [-0.36490467,  0.9066204 ,  1.6345378 , ..., -0.17773432,
        -0.03855836,  0.241436  ]], dtype=float32)

In [ ]:
test_randoms.reshape(1,-1)[0][:20]

array([-0.15535657,  0.31599703,  1.9873533 , -0.23336992, -1.2154859 ,
       -1.471258  , -0.31329656,  0.5068042 ,  0.49160695, -1.2550368 ,
        0.43182704, -1.3187023 ,  0.0042416 , -0.12583894, -0.35905623,
        1.0485922 , -0.7968093 , -0.45345792,  0.321936  ,  1.0028907 ],
      dtype=float32)

In [ ]:
testdf1 = test[:,0].reshape(-1,test_N_PATHS)
testdf1

array([[-0.15535657,  0.31599703,  1.9873533 , -0.23336992, -1.2154859 ],
       [ 0.43182704, -1.3187023 ,  0.0042416 , -0.12583894, -0.35905623],
       [-1.064633  , -1.0312779 , -0.5822    , -0.13751245, -1.8956908 ],
       ...,
       [ 1.1523024 ,  0.31871992,  0.38878578,  1.3792394 , -0.8808749 ],
       [-0.31418055, -1.366479  , -0.81390893,  1.4408053 , -0.5098722 ],
       [-0.36490467,  0.9066204 ,  1.6345378 ,  1.038103  , -1.0184551 ]],
      dtype=float32)

In [ ]:
testdf2 = test[:,1].reshape(-1,test_N_PATHS)
testdf2

array([[-1.471258  , -0.31329656,  0.5068042 ,  0.49160695, -1.2550368 ],
       [ 1.0485922 , -0.7968093 , -0.45345792,  0.321936  ,  1.0028907 ],
       [-1.991751  , -0.7618577 ,  0.09054348, -0.80207705, -0.22503746],
       ...,
       [ 0.660729  ,  0.8016318 ,  0.1350182 ,  1.156924  , -0.41682786],
       [-0.80196166, -1.9306188 , -0.02646282, -0.0627628 , -2.1372068 ],
       [-1.0469306 ,  0.7476287 , -0.17773432, -0.03855836,  0.241436  ]],
      dtype=float32)

In [ ]:
cupy.concatenate((testdf1, testdf2), axis = 1)

array([[-0.15535657,  0.31599703,  1.9873533 , ...,  0.5068042 ,
         0.49160695, -1.2550368 ],
       [ 0.43182704, -1.3187023 ,  0.0042416 , ..., -0.45345792,
         0.321936  ,  1.0028907 ],
       [-1.064633  , -1.0312779 , -0.5822    , ...,  0.09054348,
        -0.80207705, -0.22503746],
       ...,
       [ 1.1523024 ,  0.31871992,  0.38878578, ...,  0.1350182 ,
         1.156924  , -0.41682786],
       [-0.31418055, -1.366479  , -0.81390893, ..., -0.02646282,
        -0.0627628 , -2.1372068 ],
       [-0.36490467,  0.9066204 ,  1.6345378 , ..., -0.17773432,
        -0.03855836,  0.241436  ]], dtype=float32)

In [ ]:
len(test_randoms)

20000

This produces the option prices $21.22$ and $0.848$ for these two sets of option parameters in $66ms$.

It works efficiently hence we will construct an `OptionDataSet` class to wrap the above code so we can use it in Pytorch. For every `next` element, it generates uniform distributed random option parameters in the specified range, launches the GPU kernel to compute the option prices, convert the CuPy array to Pytorch tensors with zero copy via the DLPack. Note how we implemented the iterable Dataset interface:-

In [ ]:
class OptionDataSet(torch.utils.data.IterableDataset):
    
    def __init__(self, max_len=10, number_path = 1000, batch=2, threads=256,seed=15):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_BATCH = batch
        self.T = np.float32(1.0)
        self.output = cupy.zeros(self.N_BATCH*self.N_PATHS, dtype=cupy.float32) 
        self.number_of_blocks = (self.N_PATHS * self.N_BATCH - 1) // threads + 1
        self.number_of_threads = threads
        cupy.random.seed(seed)
        
    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num > self.max_length:
            raise StopIteration
        X = cupy.random.rand(self.N_BATCH, 6, dtype=cupy.float32)
        # scale the [0, 1) random numbers to the correct range for each of the option parameters
        X = X * cupy.array([200.0, 0.99, 200.0, 0.4, 0.2, 0.2], dtype=cupy.float32)
        # make sure the Barrier is smaller than the Strike price
        X[:, 1] = X[:, 0] * X[:, 1]
        randoms = cupy.random.normal(0, 1, self.N_BATCH * self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
        cupy_batched_barrier_option((self.number_of_blocks,), (self.number_of_threads,), (self.output, self.T, cupy.ascontiguousarray(X[:, 0]), 
                              cupy.ascontiguousarray(X[:, 1]), cupy.ascontiguousarray(X[:, 2]), cupy.ascontiguousarray(X[:, 3]), cupy.ascontiguousarray(X[:, 4]), cupy.ascontiguousarray(X[:, 5]), randoms, self.N_STEPS, self.N_PATHS, self.N_BATCH))
        Y = self.output.reshape(self.N_BATCH, self.N_PATHS).mean(axis=1)
        self.num += 1
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))

Put everything related to Pytorch dataset into a file `cupy_dataset.py`:-

In [ ]:
%%writefile cupy_dataset.py 
import cupy
import numpy as np
import torch
from torch.utils.dlpack import from_dlpack
cupy.cuda.set_allocator(None)

cupy_batched_barrier_option = cupy.RawKernel(r'''
extern "C" __global__ void batched_barrier_option(
    float *d_s,
    const float T,
    const float * K,
    const float * B,
    const float * S0,
    const float * sigma,
    const float * mu,
    const float * r,
    const float * d_normals,
    const long N_STEPS,
    const long N_PATHS,
    const long N_BATCH)
{
  unsigned idx =  threadIdx.x + blockIdx.x * blockDim.x;
  unsigned stride = blockDim.x * gridDim.x;
  unsigned tid = threadIdx.x;
  const float tmp3 = sqrt(T/N_STEPS);


  for (unsigned i = idx; i<N_PATHS * N_BATCH; i+=stride)
  {
    int batch_id = i / N_PATHS;
    int path_id = i % N_PATHS;
    float s_curr = S0[batch_id];
    float tmp1 = mu[batch_id]*T/N_STEPS;
    float tmp2 = exp(-r[batch_id]*T);
    unsigned n=0;
    double running_average = 0.0;
    for(unsigned n = 0; n < N_STEPS; n++){
       s_curr += tmp1 * s_curr + sigma[batch_id]*s_curr*tmp3*d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH];
       running_average += (s_curr - running_average) / (n + 1.0);
       if (running_average <= B[batch_id]){
           break;
       }
    }

    float payoff = (running_average>K[batch_id] ? running_average-K[batch_id] : 0.f); 
    d_s[i] = tmp2 * payoff;
  }
}

''', 'batched_barrier_option')

class OptionDataSet(torch.utils.data.IterableDataset):
    
    def __init__(self, max_len=10, number_path = 1000, batch=2, threads=256,seed=15):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_BATCH = batch
        self.T = np.float32(1.0)
        self.output = cupy.zeros(self.N_BATCH*self.N_PATHS, dtype=cupy.float32) 
        self.number_of_blocks = (self.N_PATHS * self.N_BATCH - 1) // threads + 1
        self.number_of_threads = threads
        cupy.random.seed(seed)
        
    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num > self.max_length:
            raise StopIteration
        X = cupy.random.rand(self.N_BATCH, 6, dtype=cupy.float32)
        # scale the [0, 1) random numbers to the correct range for each of the option parameters
        X = X * cupy.array([200.0, 0.99, 200.0, 0.4, 0.2, 0.2], dtype=cupy.float32)
        # make sure the Barrier is smaller than the Strike price
        X[:, 1] = X[:, 0] * X[:, 1]
        randoms = cupy.random.normal(0, 1, self.N_BATCH * self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
        cupy_batched_barrier_option((self.number_of_blocks,), (self.number_of_threads,), (self.output, self.T, cupy.ascontiguousarray(X[:, 0]), 
                              cupy.ascontiguousarray(X[:, 1]), cupy.ascontiguousarray(X[:, 2]), cupy.ascontiguousarray(X[:, 3]), cupy.ascontiguousarray(X[:, 4]), cupy.ascontiguousarray(X[:, 5]), randoms, self.N_STEPS, self.N_PATHS, self.N_BATCH))
        Y = self.output.reshape(self.N_BATCH, self.N_PATHS).mean(axis=1)
        self.num += 1
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))

Writing cupy_dataset.py


Here is a test code to sample 10 data points with batch size 16:-

In [ ]:
from cupy_dataset import OptionDataSet
ds = OptionDataSet(10, number_path=100000, batch=16, seed=15)
for i in ds:
    print(i[1])

tensor([1.6558e+02, 0.0000e+00, 8.0069e+01, 1.0866e+02, 7.7740e-03, 0.0000e+00,
        2.7772e+01, 0.0000e+00, 0.0000e+00, 6.4279e+01, 0.0000e+00, 5.1346e+00,
        0.0000e+00, 1.4733e+02, 4.1851e+01, 0.0000e+00], device='cuda:0')
tensor([ 57.1285,   0.0000,   0.0000, 151.9433,   0.0000,   0.0000,   0.0000,
          9.3306,   0.0000,   0.7246, 157.0885,  10.7096,   0.0000,   0.7067,
         59.1110,  14.6442], device='cuda:0')
tensor([106.4531,   0.0000,  51.1248,  12.7823,  67.4821,   0.0000,   7.3539,
          0.0000, 143.2203,  66.0655,  66.5477, 129.6811,   0.0000,  13.5559,
         27.5546,   0.0000], device='cuda:0')
tensor([4.1777e+01, 0.0000e+00, 2.5890e+00, 1.4500e+02, 0.0000e+00, 1.5099e+00,
        1.1183e+02, 5.6967e+01, 7.5751e-05, 1.2390e+01, 0.0000e+00, 3.0183e+01,
        1.3890e+01, 5.0533e+01, 3.8499e+01, 8.2232e+01], device='cuda:0')
tensor([1.0687e+02, 3.0590e+01, 8.5428e+01, 1.9835e+01, 3.0602e+01, 1.5230e+00,
        0.0000e+00, 0.0000e+00, 4.0244e+01, 0.00

We can implement the same code by using Numba to accelerate the calculation in GPU:-

In [ ]:
#%%writefile cupy_dataset.py
import numba
from numba import cuda
import cupy
import numpy as np
import math
import time
import torch
cupy.cuda.set_allocator(None)
from torch.utils.dlpack import from_dlpack

@cuda.jit
def batch_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS, N_BATCH):
    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp3 = math.sqrt(T/N_STEPS)
    #tmp2 = math.exp(-r*T)
    for i in range(ii, N_PATHS * N_BATCH, stride):
        batch_id = i // N_PATHS
        path_id = i % N_PATHS
        tmp1 = mu[batch_id]*T/N_STEPS        
        running_average = 0.0
        s_curr = S0[batch_id]
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma[batch_id]*s_curr*tmp3*d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH]
            running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average)
            #if i==0 and batch_id == 2:
                #print(s_curr)
            #if running_average <= B[batch_id]:
                #break
        d_s[i] = running_average
        #payoff = running_average - K[batch_id] if running_average > K[batch_id] else 0
        #d_s[i] = tmp2 * payoff
    

class NumbaOptionDataSet(object):
    
    def __init__(self, max_len=10, number_path = 1000, batch=2, threads=512, seed=15, T=1):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_BATCH = batch
        self.T = np.float32(T)
        self.output = cupy.zeros(self.N_BATCH*self.N_PATHS, dtype=cupy.float32) 
        self.number_of_blocks = (self.N_PATHS * self.N_BATCH - 1) // threads + 1
        self.number_of_threads = threads
        cupy.random.seed(seed)

        ############ <new
        self.Z_mean = cupy.zeros(self.N_BATCH, dtype=cupy.float32) 
        #self.Z_cov = cupy.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
        self.Z_cov = (-0.3 + cupy.random.rand(self.N_BATCH*self.N_BATCH, dtype=cupy.float32)*0.6).reshape(self.N_BATCH,self.N_BATCH)
        cupy.fill_diagonal(self.Z_cov, 1)
        ############ new>

    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num > self.max_length:
            raise StopIteration
        #X = cupy.random.rand(self.N_BATCH, 6, dtype=cupy.float32)
        # scale the [0, 1) random numbers to the correct range for each of the option parameters
        #X = X * cupy.array([200.0, 0.99, 200.0, 0.4, 0.2, 0.2], dtype=cupy.float32)
        # make sure the Barrier is smaller than the Strike price
        #X[:, 1] = X[:, 0] * X[:, 1]

        self.S0 = cupy.random.rand(self.N_BATCH, dtype=cupy.float32) * 200
        self.K = 110.0
        self.B = 100.0
        self.sigma = cupy.random.rand(self.N_BATCH, dtype=cupy.float32) * 0.4
        self.mu = cupy.random.rand(self.N_BATCH, dtype=cupy.float32) * 0.2
        self.r = 0.05

        ############ <new - add correlation between batches
        all_normals = cupy.random.multivariate_normal(self.Z_mean, self.Z_cov, 
                                              self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
        randoms = all_normals[:,0].reshape(-1,self.N_PATHS)

        for i in range(1,self.N_BATCH):
          tempdf = all_normals[:,i].reshape(-1,self.N_PATHS)
          randoms = cupy.concatenate((randoms, tempdf), axis = 1)

        randoms = randoms.reshape(1,-1)[0]
        ############ new>
        
        #randoms = cupy.random.normal(0, 1, self.N_BATCH * self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
        batch_barrier_option[(self.number_of_blocks,), (self.number_of_threads,)](self.output, self.T, self.K, self.B, self.S0, 
                                                                                  self.sigma, self.mu, self.r, randoms, self.N_STEPS, self.N_PATHS, self.N_BATCH)
        #o = self.output.reshape(self.N_BATCH, self.N_PATHS)
        #Y = o.mean(axis = 1)
        ############ <new - compute a single basket Asian option price (no barrier)
        o = np.reshape(self.output, (self.N_BATCH, self.N_PATHS))
        #print(o)
        o = o.mean(axis = 0) # average of P1, P2, ... (across batches)
        o = cupy.maximum(o - cupy.full(len(o), self.K), cupy.zeros(len(o))) # max(Avg - K, 0)
        tmp2 = math.exp(-self.r * self.T)
        Y = o.mean() * tmp2
        ############ new>

        self.num += 1

        ############ <new - combine to get X matrix
        X = cupy.array([self.K, self.B] + self.S0.tolist() +
                               self.sigma.tolist() + self.mu.tolist() + [self.r], dtype=cupy.float32)
        ############ new>

        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))

#ds = NumbaOptionDataSet(10, number_path=100000, batch=16, seed=15)
ds = NumbaOptionDataSet(batch=2)
#j = 0
for i in ds:
  print(i[0])
  #j = 0

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


tensor([1.1000e+02, 1.0000e+02, 8.5928e+01, 9.1372e+01, 3.4962e-01, 4.0304e-02,
        3.2583e-03, 1.6491e-01, 5.0000e-02], device='cuda:0')
tensor([1.1000e+02, 1.0000e+02, 1.4279e+02, 1.6435e+02, 8.9021e-02, 2.4889e-01,
        1.4426e-01, 1.3246e-01, 5.0000e-02], device='cuda:0')
tensor([1.1000e+02, 1.0000e+02, 1.3944e+02, 1.3146e+02, 6.1212e-03, 3.5014e-01,
        7.6338e-02, 9.5719e-02, 5.0000e-02], device='cuda:0')


/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


tensor([1.1000e+02, 1.0000e+02, 4.3243e+00, 1.3524e+02, 3.5992e-01, 3.9320e-01,
        5.5580e-02, 9.4486e-02, 5.0000e-02], device='cuda:0')
tensor([1.1000e+02, 1.0000e+02, 4.7965e+01, 5.6394e+01, 2.8517e-01, 1.7125e-01,
        1.5122e-01, 1.2671e-01, 5.0000e-02], device='cuda:0')
tensor([1.1000e+02, 1.0000e+02, 4.1771e+01, 6.8040e+01, 3.1539e-01, 2.6989e-02,
        6.5191e-02, 1.1657e-01, 5.0000e-02], device='cuda:0')
tensor([1.1000e+02, 1.0000e+02, 5.8224e+01, 1.0719e+02, 2.4400e-01, 7.6757e-02,
        1.5811e-01, 1.4907e-01, 5.0000e-02], device='cuda:0')
tensor([1.1000e+02, 1.0000e+02, 1.0120e+02, 6.5362e+01, 1.5767e-01, 2.3515e-01,
        1.2658e-01, 1.8852e-01, 5.0000e-02], device='cuda:0')
tensor([1.1000e+02, 1.0000e+02, 1.7604e+02, 1.8584e+02, 2.6065e-02, 2.2879e-01,
        1.1031e-01, 1.1235e-01, 5.0000e-02], device='cuda:0')
tensor([1.1000e+02, 1.0000e+02, 5.8834e+01, 1.3478e+02, 7.1777e-02, 1.8034e-01,
        1.2225e-02, 1.6500e-01, 5.0000e-02], device='cuda:0')
tensor

In [ ]:
#%%writefile cupy_dataset.py
import numba
from numba import cuda
import cupy
import numpy as np
import math
import time
import torch
cupy.cuda.set_allocator(None)
from torch.utils.dlpack import from_dlpack

@cuda.jit
def batch_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS, N_BATCH):
    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp3 = math.sqrt(T/N_STEPS)
    #tmp2 = math.exp(-r*T)
    for i in range(ii, N_PATHS * N_BATCH, stride):
        batch_id = i // N_PATHS
        path_id = i % N_PATHS
        tmp1 = mu[batch_id]*T/N_STEPS        
        running_average = 0.0
        s_curr = S0[batch_id]
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma[batch_id]*s_curr*tmp3*d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH]
            running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average)
            #if i==0 and batch_id == 2:
                #print(s_curr)
            #if running_average <= B[batch_id]:
                #break
        d_s[i] = running_average
        #payoff = running_average - K[batch_id] if running_average > K[batch_id] else 0
        #d_s[i] = tmp2 * payoff
    

class NumbaOptionDataSet(object):
    
    def __init__(self, max_len=10, number_path = 1000, batch=2, threads=512, seed=15, T=1):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_BATCH = batch
        self.T = np.float32(T)
        self.output = cupy.zeros(self.N_BATCH*self.N_PATHS, dtype=cupy.float32) 
        self.number_of_blocks = (self.N_PATHS * self.N_BATCH - 1) // threads + 1
        self.number_of_threads = threads
        cupy.random.seed(seed)

        ############ <new
        self.Z_mean = cupy.zeros(self.N_BATCH, dtype=cupy.float32) 
        #self.Z_cov = cupy.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
        self.Z_cov = (-0.3 + cupy.random.rand(self.N_BATCH*self.N_BATCH, dtype=cupy.float32)*0.6).reshape(self.N_BATCH,self.N_BATCH)
        cupy.fill_diagonal(self.Z_cov, 1)
        ############ new>

    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num > self.max_length:
            raise StopIteration
        #X = cupy.random.rand(self.N_BATCH, 6, dtype=cupy.float32)
        # scale the [0, 1) random numbers to the correct range for each of the option parameters
        #X = X * cupy.array([200.0, 0.99, 200.0, 0.4, 0.2, 0.2], dtype=cupy.float32)
        # make sure the Barrier is smaller than the Strike price
        #X[:, 1] = X[:, 0] * X[:, 1]

        self.S0 = cupy.random.rand(self.N_BATCH, dtype=cupy.float32) * 200
        self.K = 110.0
        self.B = 100.0
        self.sigma = cupy.random.rand(self.N_BATCH, dtype=cupy.float32) * 0.4
        self.mu = cupy.random.rand(self.N_BATCH, dtype=cupy.float32) * 0.2
        self.r = 0.05

        ############ <new - add correlation between batches
        all_normals = cupy.random.multivariate_normal(self.Z_mean, self.Z_cov, 
                                              self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
        randoms = all_normals[:,0].reshape(-1,self.N_PATHS)

        for i in range(1,self.N_BATCH):
          tempdf = all_normals[:,i].reshape(-1,self.N_PATHS)
          randoms = cupy.concatenate((randoms, tempdf), axis = 1)

        randoms = randoms.reshape(1,-1)[0]
        ############ new>
        
        #randoms = cupy.random.normal(0, 1, self.N_BATCH * self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
        batch_barrier_option[(self.number_of_blocks,), (self.number_of_threads,)](self.output, self.T, self.K, self.B, self.S0, 
                                                                                  self.sigma, self.mu, self.r, randoms, self.N_STEPS, self.N_PATHS, self.N_BATCH)
        #o = self.output.reshape(self.N_BATCH, self.N_PATHS)
        #Y = o.mean(axis = 1)
        ############ <new - compute a single basket Asian option price (no barrier)
        o = np.reshape(self.output, (self.N_BATCH, self.N_PATHS))
        #print(o)
        o = o.mean(axis = 0) # average of P1, P2, ... (across batches)
        o = cupy.maximum(o - cupy.full(len(o), self.K), cupy.zeros(len(o))) # max(Avg - K, 0)
        tmp2 = math.exp(-self.r * self.T)
        Y = o.mean() * tmp2
        ############ new>

        self.num += 1

        ############ <new - combine to get X matrix
        X = cupy.array([self.K, self.B] + self.S0.tolist() +
                               self.sigma.tolist() + self.mu.tolist() + [self.r], dtype=cupy.float32)
        ############ new>

        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))

#ds = NumbaOptionDataSet(10, number_path=100000, batch=16, seed=15)
ds = NumbaOptionDataSet(batch=2)
#j = 0
for i in ds:
  print(i[0])
  #j = 0

In [15]:
max_len=10
number_path = 10
batch=2
threads=512
num_stock=2
seed=15
T=1

num = 0
max_length = max_len
N_PATHS = number_path
N_STEPS = 5
N_BATCH = batch
num_stock = num_stock # new variable - by default we have 2 stocks
T = np.float32(T)
#output = cupy.zeros(N_BATCH*N_PATHS, dtype=cupy.float32) 
output = cupy.zeros(N_BATCH*N_PATHS, dtype=cupy.float32) 
number_of_blocks = (N_PATHS * N_BATCH - 1) // threads + 1
number_of_threads = threads
cupy.random.seed(seed)

Z_mean = np.zeros(2) # assume 2 options
Z_cov = [[1, 0.5],[0.5, 1]]
N_PATHS = 10
N_STEPS = 5

all_normals = cupy.random.multivariate_normal(Z_mean, Z_cov, 
         (N_PATHS, N_STEPS), dtype=cupy.float32)
# np.cov(all_normals[0,:,:][:,0],all_normals[0,:,:][:,1])
#all_normals = cupy.array(range(10*5*2), dtype=cupy.float32).reshape(10,5,2)
#all_normals[0,:,:]
print(all_normals)
#all_normals = all_normals.reshape(1,-1)[0]
print(all_normals.shape)

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


[[[-1.17146604e-01  2.57558060e+00]
  [ 1.68633729e-01  1.81585431e-01]
  [ 1.16899580e-01 -3.70912492e-01]
  [ 5.21964371e-01 -6.42374873e-01]
  [-1.23357618e+00 -2.61915028e-01]]

 [[-4.32367444e-01  8.01099241e-01]
  [ 4.68936294e-01  4.26839113e-01]
  [ 1.29471087e+00  2.12563944e+00]
  [-1.44903457e+00 -2.86948204e+00]
  [-2.60950923e-01  3.57699603e-01]]

 [[-1.03273129e+00 -1.32943892e+00]
  [-4.56799775e-01  1.51131436e-01]
  [ 1.55657065e+00  1.58704495e+00]
  [-1.03708220e+00 -1.94945049e+00]
  [ 8.93616915e-01  6.00516796e-04]]

 [[-1.56680274e+00  1.38120353e-01]
  [-7.81430602e-02 -5.18572688e-01]
  [-9.11677897e-01 -9.39698875e-01]
  [-1.34782207e+00  1.04047418e+00]
  [ 8.02706927e-03 -6.94428861e-01]]

 [[ 4.59110141e-01  9.84198987e-01]
  [ 9.67887878e-01 -9.88299370e-01]
  [-6.00035608e-01  3.79238725e-02]
  [-5.86539865e-01 -1.05388165e+00]
  [-1.31461525e+00 -1.17430723e+00]]

 [[-1.42345831e-01 -7.86623836e-01]
  [ 1.61252761e+00  1.25509393e+00]
  [ 5.17959535e-01

In [9]:
import numba
from numba import cuda
import cupy
import numpy as np
import math
import time
import torch
cupy.cuda.set_allocator(None)
from torch.utils.dlpack import from_dlpack


@cuda.jit
def batch_barrier_option(d_s, d_normals, N_STEPS, N_PATHS, N_BATCH):
    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    for i in range(ii, N_PATHS * N_BATCH, stride):
        batch_id = i // N_PATHS
        path_id = i % N_PATHS
        #running_sum = np.empty(N_STEPS), dtype=np.float64)
        for n in range(N_STEPS):
            d_s[i,n] = d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH]
            #running_sum.append(d_normals[:,n,:])
            
        #d_s[i] = sum(temp)

In [ ]:
output = np.empty((N_PATHS * N_BATCH , N_STEPS), dtype=np.float32)
batch_barrier_option[(number_of_blocks,), (number_of_threads,)](output, all_normals, N_STEPS, N_PATHS, N_BATCH)
output

In [11]:
@cuda.jit
def batch_barrier_option(d_s, d_normals, N_STEPS, N_PATHS, N_BATCH):
    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    for i in range(ii, N_PATHS, stride):
        #batch_id = i // N_PATHS
        #path_id = i % N_PATHS
        #running_sum = np.empty(N_STEPS), dtype=np.float64)
        for n in range(N_STEPS):
            for j in range(N_BATCH):
              d_s[i*N_STEPS+n,j] = d_normals[i, n, j]
            #d_s[i,n+1] = d_normals[i, n, 1]
            #running_sum.append(d_normals[:,n,:])
        #all_normals[0,0,:]    
        #d_s[i] = sum(temp)

In [20]:
all_normals = cupy.array(range(10*5*2), dtype=cupy.float32).reshape(10,5,2)
output = np.zeros((N_PATHS * N_STEPS, N_BATCH), dtype=np.float32)
batch_barrier_option[(number_of_blocks,), (number_of_threads,)](output, all_normals, N_STEPS, N_PATHS, N_BATCH)
output

array([[ 0.,  1.],
       [ 2.,  3.],
       [ 4.,  5.],
       [ 6.,  7.],
       [ 8.,  9.],
       [10., 11.],
       [12., 13.],
       [14., 15.],
       [16., 17.],
       [18., 19.],
       [20., 21.],
       [22., 23.],
       [24., 25.],
       [26., 27.],
       [28., 29.],
       [30., 31.],
       [32., 33.],
       [34., 35.],
       [36., 37.],
       [38., 39.],
       [40., 41.],
       [42., 43.],
       [44., 45.],
       [46., 47.],
       [48., 49.],
       [50., 51.],
       [52., 53.],
       [54., 55.],
       [56., 57.],
       [58., 59.],
       [60., 61.],
       [62., 63.],
       [64., 65.],
       [66., 67.],
       [68., 69.],
       [70., 71.],
       [72., 73.],
       [74., 75.],
       [76., 77.],
       [78., 79.],
       [80., 81.],
       [82., 83.],
       [84., 85.],
       [86., 87.],
       [88., 89.],
       [90., 91.],
       [92., 93.],
       [94., 95.],
       [96., 97.],
       [98., 99.]], dtype=float32)

In [ ]:
temp

In [ ]:
(0.0)

In [ ]:
cupy.sum(all_normals, axis=2)

array([[ 0.12155032,  0.12178636, -0.15841234, -0.22159326, -0.35646957],
       [ 0.22381158,  0.2510133 ,  0.05132556,  0.0307359 , -0.18171096],
       [ 0.01105827,  0.10306579, -0.08766937, -0.04900521, -0.04341554],
       [ 0.2407403 ,  0.32266536, -0.02087653, -0.09617114, -0.03780007],
       [ 0.01496691, -0.15707135,  0.17004573, -0.12790179,  0.06761813],
       [ 0.13263252, -0.11955708, -0.0942567 ,  0.20174909, -0.24877626],
       [-0.2681654 , -0.0755499 , -0.02301556,  0.08088765,  0.23512149],
       [-0.29130742,  0.052378  , -0.08673185,  0.04828384, -0.12331009],
       [-0.01231092,  0.04433234, -0.10872042, -0.16182828,  0.15016949],
       [-0.05149448,  0.14565265, -0.02566338,  0.0285185 ,  0.15511928]],
      dtype=float32)

In [ ]:
cupy.cov(all_normals[:,1,1])

array(0.65482279)

In [ ]:
randoms = all_normals[:,0].reshape(-1,self.N_PATHS)

In [ ]:
Z = (0.8 + cupy.random.rand(5*5, dtype=cupy.float32)*0.2).reshape(5,5)
cupy.fill_diagonal(Z, 1)
Z

array([[1.        , 0.84962684, 0.914959  , 0.9977649 , 0.8646952 ],
       [0.8481131 , 1.        , 0.9628569 , 0.897008  , 0.8974829 ],
       [0.82393116, 0.9581161 , 1.        , 0.86098695, 0.9111912 ],
       [0.9032356 , 0.9038602 , 0.81335443, 1.        , 0.87922835],
       [0.9953061 , 0.8087999 , 0.9703542 , 0.9417764 , 1.        ]],
      dtype=float32)

In [ ]:
import numba
from numba import cuda
import cupy
import numpy as np
import math
import time
import torch
cupy.cuda.set_allocator(None)
from torch.utils.dlpack import from_dlpack

@cuda.jit
def batch_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS, N_BATCH):
    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp3 = math.sqrt(T/N_STEPS)
    #tmp2 = math.exp(-r*T)
    for i in range(ii, N_PATHS * N_BATCH, stride):
        batch_id = i // N_PATHS
        path_id = i % N_PATHS
        tmp1 = mu[batch_id]*T/N_STEPS        
        running_average = 0.0
        s_curr = S0[batch_id]
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma[batch_id]*s_curr*tmp3*d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH]
            running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average)
            #if i==0 and batch_id == 2:
                #print(s_curr)
            #if running_average <= B[batch_id]:
                #break
        d_s[i] = running_average
        #payoff = running_average - K[batch_id] if running_average > K[batch_id] else 0
        #d_s[i] = tmp2 * payoff
    

class NumbaOptionDataSet(object):
    
    def __init__(self, max_len=10, number_path = 1000, batch=2, threads=512, num_stock=2, seed=15, T=1):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_BATCH = batch
        self.num_stock = num_stock # new variable - by default we have 2 stocks
        self.T = np.float32(T)
        self.output = cupy.zeros(self.N_BATCH*self.N_PATHS, dtype=cupy.float32) 
        self.number_of_blocks = (self.N_PATHS * self.N_BATCH - 1) // threads + 1
        self.number_of_threads = threads
        cupy.random.seed(seed)

        ############ <new
        self.Z_mean = cupy.array([0.0, 0.0], dtype=cupy.float32)
        self.Z_cov = cupy.array([[1.0, 0.5],[0.5, 1.0]], dtype=cupy.float32)
        ############ new>
        
    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num > self.max_length:
            raise StopIteration
        X = cupy.random.rand(self.N_BATCH, 3 * self.num_stock + 3, dtype=cupy.float32)
        #X = cupy.random.rand(self.N_BATCH, 6, dtype=cupy.float32)
        # scale the [0, 1) random numbers to the correct range for each of the option parameters
        X = X * cupy.array([200.0, 0.99] + [200.0]*self.num_stock + [0.4]*self.num_stock + 
                           [0.2]*self.num_stock + [0.2], dtype=cupy.float32) #K, B, 50S0, 50sigma, 50mu, r
        # make sure the Barrier is smaller than the Strike price
        X[:, 1] = X[:, 0] * X[:, 1]

        ############ <new - add correlation between underlyings
        all_normals = cupy.random.multivariate_normal(self.Z_mean, self.Z_cov, 
                                              self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
        randoms = all_normals[:,0].reshape(-1,self.N_PATHS)

        for i in range(1,self.num_stock):
          tempdf = all_normals[:,i].reshape(-1,self.N_PATHS)
          randoms = cupy.concatenate((randoms, tempdf), axis = 1)

        randoms = randoms.reshape(1,-1)[0]
        ############ new>
        
        #randoms = cupy.random.normal(0, 1, self.N_BATCH * self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
        batch_barrier_option[(self.number_of_blocks,), (self.number_of_threads,)](self.output, self.T, X[:, 0], 
                              X[:, 1], X[:, 2:2+self.num_stock], X[:, 2+self.num_stock:2+2*self.num_stock],
                              X[:, 2+2*self.num_stock:2+3*self.num_stock], X[:, 5], randoms, self.N_STEPS, self.N_PATHS, self.N_BATCH)
        #o = self.output.reshape(self.N_BATCH, self.N_PATHS)
        #Y = o.mean(axis = 1)
        ############ <new - compute a single basket Asian option price (no barrier)
        o = np.reshape(self.output, (self.N_BATCH, self.N_PATHS))
        o = o.mean(axis = 0) # average of P1, P2, ... (across batches)
        o = cupy.maximum(o - cupy.full(len(o), self.K), cupy.zeros(len(o))) # max(Avg - K, 0)
        tmp2 = math.exp(-self.r * self.T)
        Y = o.mean() * tmp2
        ############ new>

        self.num += 1

        ############ <new - combine to get X matrix
        X = cupy.column_stack((cupy.full(len(self.S0), self.K).T, cupy.full(len(self.S0), self.B).T, self.S0.T,
                               self.sigma.T, self.mu.T, cupy.full(len(self.S0), self.r).T))
        ############ new>

        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))

#ds = NumbaOptionDataSet(10, number_path=100000, batch=16, seed=15)
ds = NumbaOptionDataSet()
for i in ds:
  print(i[1])

In [ ]:
max_len=10
number_path = 1000
batch=2
threads=512
num_stock=2
seed=15
T=1

num = 0
max_length = max_len
N_PATHS = number_path
N_STEPS = 365
N_BATCH = batch
num_stock = num_stock # new variable - by default we have 2 stocks
T = np.float32(T)
output = cupy.zeros(N_BATCH*N_PATHS, dtype=cupy.float32) 
number_of_blocks = (N_PATHS * N_BATCH - 1) // threads + 1
number_of_threads = threads
cupy.random.seed(seed)

Z_mean = cupy.array([0.0, 0.0], dtype=cupy.float32)
Z_cov = cupy.array([[1.0, 0.5],[0.5, 1.0]], dtype=cupy.float32)

In [ ]:
X = cupy.random.rand(N_BATCH, 3 * num_stock + 3, dtype=cupy.float32)
#X = cupy.random.rand(self.N_BATCH, 6, dtype=cupy.float32)
# scale the [0, 1) random numbers to the correct range for each of the option parameters
X = X * cupy.array([200.0, 0.99] + [200.0]*num_stock + [0.4]*num_stock + 
                    [0.2]*num_stock + [0.2], dtype=cupy.float32) #K, B, 50S0, 50sigma, 50mu, r
# make sure the Barrier is smaller than the Strike price
X[:, 1] = X[:, 0] * X[:, 1]
X

array([[1.1365630e+01, 8.1278534e+00, 1.2313335e+02, 2.9515425e+01,
        3.0960795e-01, 2.2900939e-01, 7.0528589e-02, 1.4074355e-01,
        4.7650620e-02],
       [1.7088293e+02, 9.2240868e+01, 1.6616214e+02, 1.5265572e+02,
        2.4826026e-01, 5.2797593e-02, 1.7911892e-01, 1.7636879e-01,
        9.1669662e-03]], dtype=float32)

In [ ]:
X[:, 2:2+num_stock]
X[:, 2+num_stock:2+2*num_stock]
X[:, 2+2*num_stock:2+3*num_stock]

array([[0.07052859, 0.14074355],
       [0.17911892, 0.17636879]], dtype=float32)

In [ ]:
max_len=10
number_path = 5
batch=2
threads=512
seed=15
T=1
Z_mean=cupy.array([0.0, 0.0], dtype=cupy.float32)
Z_cov=cupy.array([[1.0, 0.1],[0.1, 1.0]], dtype=cupy.float32)
K=cupy.array([110.0, 120.0], dtype=cupy.float32)
B=cupy.array([100.0, 90.0], dtype=cupy.float32)
S0=cupy.array([120.0, 100.0], dtype=cupy.float32)
sigma=cupy.array([0.35, 0.2], dtype=cupy.float32)
mu=cupy.array([0.15, 0.1], dtype=cupy.float32)
r=cupy.array([0.05, 0.05], dtype=cupy.float32)

num = 0
max_length = max_len
N_PATHS = number_path
N_STEPS = 365
N_BATCH = batch
T = np.float32(T)
output = cupy.zeros(N_BATCH*N_PATHS, dtype=cupy.float32) 
number_of_blocks = (N_PATHS * N_BATCH - 1) // threads + 1
number_of_threads = threads
cupy.random.seed(seed)

all_normals = cupy.random.multivariate_normal(Z_mean, Z_cov, 
                                              N_PATHS * N_STEPS, dtype=cupy.float32)
randoms = all_normals[:,0].reshape(-1,N_PATHS)

for i in range(1,N_BATCH):
  tempdf = all_normals[:,i].reshape(-1,N_PATHS)
  randoms = cupy.concatenate((randoms, tempdf), axis = 1)

randoms = randoms.reshape(1,-1)[0]

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')


In [ ]:
batch_barrier_option[(number_of_blocks,), (number_of_threads,)](output, T, K, B, S0, sigma, mu, r, randoms, N_STEPS, N_PATHS, N_BATCH)

In [ ]:
s_curr = S0[1]
s_curr

array(100., dtype=float32)

In [ ]:
cupy.append(s_curr, 0.2)

array([100. ,   0.2])

In [ ]:
import numba
from numba import cuda
import pandas as pd

In [ ]:
K=cupy.array([110.0, 120.0], dtype=cupy.float32)
B=cupy.array([100.0, 90.0], dtype=cupy.float32)
S0=cupy.array([120.0, 100.0], dtype=cupy.float32)
sigma=cupy.array([0.35, 0.2], dtype=cupy.float32)
mu=cupy.array([0.15, 0.1], dtype=cupy.float32)
r=cupy.array([0.05, 0.05], dtype=cupy.float32)

In [ ]:
X = cupy.column_stack((K.T,B.T,S0.T,sigma.T,mu.T,r.T))
X

array([[1.1e+02, 1.0e+02, 1.2e+02, 3.5e-01, 1.5e-01, 5.0e-02],
       [1.2e+02, 9.0e+01, 1.0e+02, 2.0e-01, 1.0e-01, 5.0e-02]],
      dtype=float32)

In [ ]:
from_dlpack(X.toDlpack())

tensor([[1.1000e+02, 1.0000e+02, 1.2000e+02, 3.5000e-01, 1.5000e-01, 5.0000e-02],
        [1.2000e+02, 9.0000e+01, 1.0000e+02, 2.0000e-01, 1.0000e-01, 5.0000e-02]],
       device='cuda:0')

### Model
To map the option parameters to price, we use 6 layers of fully connected neural network with hidden dimension 512 as inspired by [this paper](https://arxiv.org/abs/1809.02233). Writing this DL price model into a file `model.py`:-

In [ ]:
%%writefile model.py
import torch.nn as nn
import torch.nn.functional as F
import torch


class Net(nn.Module):

    def __init__(self, hidden=1024):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, hidden)
        self.fc5 = nn.Linear(hidden, hidden)
        self.fc6 = nn.Linear(hidden, 1)
        self.register_buffer('norm',
                             torch.tensor([200.0,
                                           198.0,
                                           200.0,
                                           0.4,
                                           0.2,
                                           0.2]))

    def forward(self, x):
        # normalize the parameter to range [0-1] 
        x = x / self.norm
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = F.elu(self.fc4(x))
        x = F.elu(self.fc5(x))
        return self.fc6(x)

Writing model.py


As we know the random parameters' scaling factors, the input parameters are first scaled back to a range of (0-1) by dividing them by (200.0, 198.0, 200.0, 0.4, 0.2, 0.2). Then they are projected 5 times to the hidden dimension of 512 after the `ELu` activation function. `ELu` is chosen because we need to compute the second order differentiation of the parameters. If use ReLu, the second order differentiation will always be zero. The last layer is a linear layer that maps the hidden dimension to the predicted option price. 

For training, we use [Ignite](https://github.com/pytorch/ignite) which is a high-level library to train neural networks in PyTorch. We use `MSELoss` as the loss function, `Adam` as the optimizer and `CosineAnnealingScheduler` as the learning rate scheduler. The following code is feeding the random option data to the pricing model to train it.

In [ ]:
!pip install pytorch-ignite

     |████████████████████████████████| 204kB 13.9MB/s 


In [ ]:
from ignite.engine import Engine, Events
from ignite.handlers import Timer
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from model import Net
from cupy_dataset import NumbaOptionDataSet
timer = Timer(average=True)
model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
dataset = NumbaOptionDataSet(max_len=100, number_path = 1024, batch=2)
#dataset = OptionDataSet(max_len=10000, number_path = 1024, batch=4800)

def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    y_pred = model(x)
    loss = loss_fn(y_pred[:,0], y)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 100

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output, 'average time', timer.value())
        
trainer.run(dataset, max_epochs=100)

/usr/local/lib/python3.7/dist-packages/cupy/random/_distributions.py:476: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.7/dist-packages/cupy/random/_generator.py:337: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  _util.experimental('cupy.random.RandomState.multivariate_normal')
Current run is terminating due to exception: expected scalar type Float but found Double
Engine run is terminating due to exception: expected scalar type Float but found Double


RuntimeError: ignored

The loss is keeping decreasing which means the pricing model can predict the option prices better. It takes about $12ms$ to compute one mini-batch in average, In the following sections, we will try to expore the full potentials of the GPU to accelerate the training.

### TensorCore mixed precision training

The V100 GPUs have 640 tensor cores that can accelerate half precision matrix multiplication calculation which is the core computation done by the DL model. [Apex library](https://github.com/NVIDIA/apex) developed by NVIDIA makes mixed precision and distributed training in Pytorch easy. By changing 3 lines of code, it can use the tensor cores to accelerate the training. 

In [ ]:
!git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 8038, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 8038 (delta 58), reused 68 (delta 29), pack-reused 7913
Receiving objects: 100% (8038/8038), 14.11 MiB | 23.64 MiB/s, done.
Resolving deltas: 100% (5457/5457), done.


In [ ]:
cd apex

/content/apex


In [ ]:
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-rdfqqr3k
Created temporary directory: /tmp/pip-req-tracker-bhgvya29
Created requirements tracker '/tmp/pip-req-tracker-bhgvya29'
Created temporary directory: /tmp/pip-install-fr7sbd60
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-kjeiz50m
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-bhgvya29'
    Running setup.py (path:/tmp/pip-req-build-kjeiz50m/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.8.1+cu101


    running egg_info
    creating /tmp/pip-req-build-kjeiz50m/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-kjeiz50m/pip-egg-info/apex.egg-info/PK

In [ ]:
from apex import amp
from ignite.engine import Engine, Events
from torch.nn import MSELoss
from ignite.handlers import Timer
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from model import Net
from cupy_dataset import OptionDataSet
timer = Timer(average=True)
model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
# set the AMP optimization level to O1
opt_level = 'O1'
# wrap the optimizer and model
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
dataset = OptionDataSet(max_len=100, number_path = 1024, batch=48)

def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    y_pred = model(x)
    loss = loss_fn(y_pred[:,0], y)
    # amp handles the auto loss scaling
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 100
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output, 'average time', timer.value())
        
trainer.run(dataset, max_epochs=100)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 40

State:
	iteration: 10000
	epoch: 100
	epoch_length: 100
	max_epochs: 100
	output: 0.17109763622283936
	batch: <class 'tuple'>
	metrics: <class 'dict'>
	dataloader: <class 'cupy_dataset.OptionDataSet'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

It improves to compute each mini-batch in $8ms$. As we reduce the model weights to half precision for better performance, the loss need to be scaled to make sure the half precision dynamic range aligns with the computation. It is guessing what is the correct loss scaling factor and adjust it automatically if the gradient overflows. In the end, we will get the best hardware acceleration while maintaining the accuracy of model prediction.

### Multiple GPU training

Apex makes multiple GPU training easy. Working on the same training script, we need to take care of a few extra steps:

1. Add the argument `--local_rank` which will be automatically set by the distributed launcher
2. Initialize the process group
2. Generate independent batched data based on process id in the dataset.
3. Wrap the model and optimizer to handle distributed computation. 
4. Scale the loss and optimizer

To launch distributed training, we need to put everything into a python file. Following is an example:-

In [ ]:
pwd

'/content/apex'

In [ ]:
cd ..

/


In [ ]:
%%writefile distributed_train.py 
import cupy
import numpy as np
import math
import time
import os
import torch
from torch.utils.dlpack import from_dlpack
import torch.nn as nn
import torch.nn.functional as F
import torch
from apex import amp
from ignite.engine import Engine, Events
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from apex.parallel import DistributedDataParallel 
import argparse
from model import Net
from cupy_dataset import OptionDataSet

parser = argparse.ArgumentParser()
parser = argparse.ArgumentParser()
# this local_rank arg is automaticall set by distributed launch
parser.add_argument("--local_rank", default=0, type=int)
args = parser.parse_args()

args.distributed = False
if 'WORLD_SIZE' in os.environ:
    args.distributed = int(os.environ['WORLD_SIZE']) > 1

if args.distributed:
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend='nccl',
                                         init_method='env://')

torch.backends.cudnn.benchmark = True


model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
opt_level = 'O1'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
if args.distributed:
    model = DistributedDataParallel(model)
dataset = OptionDataSet(max_len=10000, number_path = 1024, batch=10240, seed=args.local_rank)

def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    y_pred = model(x)
    loss = loss_fn(y_pred[:,0], y)
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 100

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output)
        
trainer.run(dataset, max_epochs=100)

Overwriting distributed_train.py


To launch multiple processes training, we need to run the following command:-

In [ ]:
%reset -f

!python -m torch.distributed.launch --nproc_per_node=4 distributed_train.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Traceback (most recent call last):
  File "distributed_train.py", line 11, in <module>
    from apex import amp
ModuleNotFoundError: No module named 'apex'
Traceback (most recent call last):
  File "distributed_train.py", line 11, in <module>
    from apex import amp
ModuleNotFoundError: No module named 'apex'
Traceback (most recent call last):
  File "distributed_train.py", line 11, in <module>
    from apex import amp
ModuleNotFoundError: No module named 'apex'
Traceback (most recent call last):
  File "distributed_train.py", line 11, in <module>
    from apex import amp
ModuleNotFoundError: No module named 'apex'
Killing subprocess 290
Killing subprocess 291
Killing subprocess 292
Killing subpr

It works and all the GPUs are busy to train this network. However, it has a few problems:-
   
    1. There is no model serialization so the trained model is not saved
    2. There is no validation dataset to check the training progress
    3. Most of the time is spent in Monte Carlo simulation hence the training is slow
    4. We use a few paths(1024) for each option parameter set which is noise and the model cannot converge to a low cost value.
We will address these problems in the next notebook